In [1]:
import pickle
import z_update_layer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten
import numpy as np
import tensorflow as tf
import tensorflow.keras.optimizers


In [2]:
import pickle
with open("/home/richertlr/Documents/summer2019/cifar-10-batches-py/data_batch_1",'rb') as fo:
    dict=pickle.load(fo,encoding="bytes")

In [3]:
with open("/home/richertlr/Documents/summer2019/cifar-10-batches-py/test_batch","rb") as fo2:
    dict2=pickle.load(fo2,encoding="bytes")

In [4]:
temp = np.reshape(dict[b'data']/255,[10000,3,32,32])
x_input = np.transpose(temp,(0,2,3,1))
temp = np.reshape(dict2[b'data']/255,[-1,3,32,32])
x_val = np.transpose(temp,(0,2,3,1))

In [5]:
labels = to_categorical(dict[b'labels'])
labels_val = to_categorical(dict2[b'labels'])

In [6]:
#model = Sequential()
#model.add(z_update_layer.multilayerADMMsparseCodingTightFrame([32,32],[[5,5],[4,4]],[[2,2]],1.,.2,24,input_shape=(None,32,32,3)))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid'))
#model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=[3,3],strides=[1,1],padding='valid',activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid'))
#model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=[3,3],strides=[1,1],padding='valid',activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid'))
#model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=[3,3],strides=[1,1],padding='valid',activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid'))
#model.add(Flatten())
#model.add(Dense(10,activation='softmax'))

In [7]:
inputLayer = tensorflow.keras.Input(shape=(32,32,3))
x,reconInputLayer = z_update_layer.multilayerADMMsparseCodingTightFrame([32,32],[[5,5],[4,4]],[[2,2]],1.,.2,5)(inputLayer)
x2 = tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid')(x)
x3 = tf.keras.layers.Conv2D(filters=32,kernel_size=[3,3],strides=[1,1],padding='valid',activation='relu')(x2)
x4 = tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid')(x3)
x5 = tf.keras.layers.Conv2D(filters=32,kernel_size=[3,3],strides=[1,1],padding='valid',activation='relu')(x4)
x6 = tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid')(x5)
x7 = tf.keras.layers.Conv2D(filters=32,kernel_size=[3,3],strides=[1,1],padding='valid',activation='relu')(x6)
x8 = tf.keras.layers.MaxPooling2D(pool_size=[2,2],strides=[1,1],padding='valid')(x7)
x9 = Flatten()(x8)
y = Dense(10,activation='softmax')(x9)

model = Model(inputs=inputLayer,outputs=y)
#model.add_loss(0.00001*z_update_layer.cropped_l2_loss(inputLayer,reconInputLayer))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
#tf.GraphKeys.LOSSES

In [ ]:
sgd = tf.keras.optimizers.SGD(lr=0.0001, decay=0.000125, momentum=0.5, nesterov=True)
model.compile(optimizer='sgd', loss='categorical_crossentropy')
model.fit(x_input, labels, epochs=16, batch_size=32, validation_data=(x_val,labels_val))

Train on 10000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/16
 9984/10000 [============================>.] - ETA: 2s - loss: 14.4612

In [15]:
a = tf.constant([1,2,3])

In [8]:
a = tensorflow.constant([1,2,3])

NameError: name 'tensorflow' is not defined

In [9]:
import tensorflow
a = tensorflow.constant([1,2,3])

In [12]:
sess = Session()
sess.run(a*2)

array([2, 4, 6], dtype=int32)